In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import itertools
import numpy as np
import math

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [82]:
# Create a list of all possible combinations of 0, 1, and 2
vals = [0, 1, 2]
combinations = list(itertools.product(vals, repeat=9))

# Filter out the combinations where the middle element is not 0
filtered_combinations = list(filter(lambda x: x[4] == 1, combinations))

# Reshape the filtered combinations to tensors of shape (3x3)
tensors = list(set([torch.tensor(combination).reshape(3, 3) for combination in filtered_combinations]))

In [83]:
len(tensors)

6561

In [72]:
prey_states = generate_prey_states()

In [74]:
len(prey_states)

5265

In [73]:
prey_states

[tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 0, 1]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 0, 2]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 1, 0]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 1, 1]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 1, 2]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 2, 0]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 2, 1]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 2, 2]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 0, 0]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 0, 1]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 0, 2]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 1, 0]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 1, 1]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 1, 2]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [1, 2, 0]]),
 tensor(

In [13]:
def generate_random_matrix(values, size, p):
    matrix = torch.tensor(np.random.choice([0, 1, 2], size=(5, 5), p=[0.9, 0.08, 0.02]), dtype=torch.float32)
    matrix[2,2] = 2
    return matrix

def find_closest_cell(t, point, n):
    """
    t: torch tensor filled with 1s and 2s
    point: tuple (x,y) of the currently observed point
    n: number to search for (1 or 2)
    """
    closest_dist = math.inf
    closest_cell = None
    
    cells = [(i, j) for i in range(t.size()[0]) for j in range(t.size()[1]) if t[i][j] == n]
    
    for cell_point in cells:
        dist = abs(point[0] - cell_point[0]) + abs(point[1] - cell_point[1]) 
        if dist < closest_dist:
            closest_dist = dist
            closest_cell = cell_point
                    
    if closest_cell is None:
        return math.inf
    else:
        return closest_dist
    
def find_best_move_predator(t, player):
    """
    Should return list of probabilities
    """
    n = 1 if player == 2 else 2
    best_points = []
    best_distance = math.inf

    for point in [[(1,2), 0], [(2,3), 1], [(3,2), 2], [(2,1), 3]]:
        dist = find_closest_cell(t, point[0], n)

        if dist < best_distance:
            best_distance = dist
            best_points = [point]
        elif dist == best_distance:
            best_points.append(point)

    return np.random.choice([p[1] for p in best_points])

In [15]:
random_matrices = set()

new_matrices = [generate_random_matrix([0,1,2], 1, (3,3), [0.9, 0.08, 0.02]) for _ in range(1000)]

while len(random_matrices) < 1000:
    random_matrices.update()

labels = [find_best_move_predator(t) for t in random_matrices]  

In [55]:
random_matrices = [generate_random_matrix([0,1,2], 1, (3,3), [0.9, 0.08, 0.02]) for _ in range(1000)]
random_matrices[0]

tensor([[0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [61]:
a = {1,2,3}
a.update([4])
a

{1, 2, 3, 4}

In [56]:
a = dict()
a[random_matrices[1]] = 10

In [5]:
from torch.utils.data import Dataset, DataLoader

class MatrixDataset(Dataset):
    def __init__(self, matrices, labels):
        self.matrices = matrices
        self.labels = labels

    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        matrix = self.matrices[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return matrix, label

batch_size = 32
dataset = MatrixDataset(random_matrices, labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(25, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 4)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

num_epochs = 300

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 1.3869602121412754
Epoch: 2, Loss: 1.3698819316923618
Epoch: 3, Loss: 1.3538994044065475
Epoch: 4, Loss: 1.3331749513745308
Epoch: 5, Loss: 1.3085955791175365
Epoch: 6, Loss: 1.2840671837329865
Epoch: 7, Loss: 1.267162922769785
Epoch: 8, Loss: 1.2500368654727936
Epoch: 9, Loss: 1.2304236963391304
Epoch: 10, Loss: 1.2192141078412533
Epoch: 11, Loss: 1.198549434542656
Epoch: 12, Loss: 1.1789575815200806
Epoch: 13, Loss: 1.163257721811533
Epoch: 14, Loss: 1.1596285421401262
Epoch: 15, Loss: 1.142360720783472
Epoch: 16, Loss: 1.1094874422997236
Epoch: 17, Loss: 1.0988751705735922
Epoch: 18, Loss: 1.0673315171152353
Epoch: 19, Loss: 1.072192883118987
Epoch: 20, Loss: 1.0393913928419352
Epoch: 21, Loss: 1.0137258917093277
Epoch: 22, Loss: 1.0033973399549723
Epoch: 23, Loss: 0.9673689510673285
Epoch: 24, Loss: 0.9568778835237026
Epoch: 25, Loss: 0.9280357267707586
Epoch: 26, Loss: 0.9058035463094711
Epoch: 27, Loss: 0.9006059058010578
Epoch: 28, Loss: 0.8717989455908537
Epoch:

In [9]:
# Switch the network to evaluation mode
net.eval()

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=25, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

In [23]:
input_matrix = generate_random_matrix().unsqueeze(0).to(device)
input_matrix

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 2., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]]], device='cuda:0')

In [24]:
# Make predictions
with torch.no_grad():  # Disable gradient calculation to save memory and computation
    logits = net(input_matrix)
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)

print("Predicted class:", predicted_class.item())
print("Class probabilities:", probabilities.cpu().numpy())

Predicted class: 1
Class probabilities: [[2.4268265e-06 9.9954742e-01 4.5013209e-04 6.7772468e-09]]
